In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import avg
from pyspark.sql.functions import expr
import pyspark.sql.functions as F


In [2]:
spark = SparkSession.builder.appName("SPARK_TITANIC_01").getOrCreate()

23/12/10 11:18:14 WARN Utils: Your hostname, Sivas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
23/12/10 11:18:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/10 11:18:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("titanic_dataset.csv")

In [4]:
# df.show(200)
df.count()

891

In [5]:
# Filter Data which has Null Age
filtered_df = df.filter("Age is not null")


In [5]:
filtered_df.select('*').where(filtered_df.PassengerId == '295').show()
# filtered_df.count()

+-----------+--------+------+----------------+----+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|            Name| Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+----------------+----+----+-----+-----+------+------+-----+--------+
|        295|       0|     3|Mineff, Mr. Ivan|male|24.0|    0|    0|349233|7.8958| NULL|       S|
+-----------+--------+------+----------------+----+----+-----+-----+------+------+-----+--------+


In [6]:
filtered_df.select('*').where(filtered_df.Ticket == 'A/5 21171').show()

+-----------+--------+------+--------------------+----+----+-----+-----+---------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|   Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|A/5 21171|7.25| NULL|       S|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+----+-----+--------+


In [29]:
# Determine the Total Death and Survived.
# filtered_df.where(filtered_df.Survived == 0).count()
filtered_df.where(filtered_df.Survived > 0).collect()

[Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S'),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S'),
 Row(PassengerId=9, Survived=1, Pclass=3, Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age=27.0, SibSp=0, Parch=2, Ticket='347742', Fare=11.1333, Cabin=None, Embarked='S'),
 Row(PassengerId=10, Survived=1, Pclass=2, Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age=14.0, SibSp=1, Parch=0, Ticket='237736', Fare=30.0708, Cabin=None, Embarked='C'),
 Row(PassengerId=11, Survived=1, 

In [28]:
(filtered_df.select('Embarked','Sex')
 .where(col('Survived')
 .isNotNull and col('Survived') == 0)
 .groupby( filtered_df.Embarked,filtered_df.Sex)
 .count()
 .orderBy("Embarked","count",ascending=True)
 .explain())

AttributeError: 'GroupedData' object has no attribute 'collect'

In [7]:
# Define a Schema Up-Front For Incomming data.
titanic_schema = StructType([
            StructField("passenger_id", IntegerType(), True),
            StructField("survived", IntegerType(), True),
            StructField("passenger_class", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("sex", StringType(), True),
            StructField("age ", FloatType(), True),
            StructField("sib_spouse", IntegerType(), True),
            StructField("parerent_child", IntegerType(), True),
            StructField("tickets", StringType(), True),
            StructField("fare", FloatType(), True),
            StructField("embarked", StringType(), True),
                            ])   

In [8]:
# Drop The Columns "Cabin"
after_Filtered_df = filtered_df.drop("Cabin")
# after_Filtered_df.printSchema()

In [ ]:
# Create A DataFrame Using The Above Schema.
new_dataframe =  spark.createDataFrame(data= after_Filtered_df.collect(), schema=titanic_schema)
# new_dataframe.show(20)


In [ ]:
# new_dataframe.explain()

In [19]:
# Operations with Spark DataFrame Columns.
# Adding a New Column using a conditional Expression.

# Determine the Average Fare.
avgFareRate = new_dataframe.select(avg(new_dataframe.fare)).collect().__str__()
avg1 = avgFareRate.split('=', 2)
delimeter = [')]']
temp_data = []
for d in delimeter:
    for item in avg1:
        temp_data.extend(item.split(d))
        
# print(temp_data)

def is_float(string):
    try:
        # Return true if float
        float(string)
        return True
    except ValueError:
        # Return False if Error
        return False
    
averageTicketRate = 0.0
for data in temp_data:
    if  is_float(data):
        averageTicketRate = data
# print(averageTicketRate)  

# Explain the Spark Plans:
new_dataframe.withColumn('Expensive', expr("Fare >" + averageTicketRate)).show()



# new_dataframe.show()

+------------+--------+---------------+--------------------+------+----+----------+--------------+----------------+-------+--------+---------+
|passenger_id|survived|passenger_class|                name|   sex|age |sib_spouse|parerent_child|         tickets|   fare|embarked|Expensive|
+------------+--------+---------------+--------------------+------+----+----------+--------------+----------------+-------+--------+---------+
|           1|       0|              3|Braund, Mr. Owen ...|  male|22.0|         1|             0|       A/5 21171|   7.25|       S|    false|
|           2|       1|              1|Cumings, Mrs. Joh...|female|38.0|         1|             0|        PC 17599|71.2833|       C|     true|
|           3|       1|              3|Heikkinen, Miss. ...|female|26.0|         0|             0|STON/O2. 3101282|  7.925|       S|    false|
|           4|       1|              1|Futrelle, Mrs. Ja...|female|35.0|         1|             0|          113803|   53.1|       S|     true|

In [35]:
aqx = new_dataframe.groupBy('age ').count()
aqx.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[age #233], functions=[count(1)])
   +- Exchange hashpartitioning(age #233, 200), ENSURE_REQUIREMENTS, [plan_id=626]
      +- HashAggregate(keys=[knownfloatingpointnormalized(normalizenanandzero(age #233)) AS age #233], functions=[partial_count(1)])
         +- Project [age #233]
            +- Scan ExistingRDD[passenger_id#228,survived#229,passenger_class#230,name#231,sex#232,age #233,sib_spouse#234,parerent_child#235,tickets#236,fare#237,embarked#238]


In [ ]:
sdx = new_dataframe.select('passenger_id','name')
# sdx.show()

Narrow Transformations

filter rows where Age = 10
add a new column: adding first_name and last_name
alter an exisitng column: adding 5 to age column
select relevant columns

In [39]:
df_narrow_tansform  =(
    new_dataframe
    .withColumn('first_name', F.split('name', ', ').getItem(1))
    .withColumn('last_name', F.split('name', ', ').getItem(0))
    .select('*')
)

df_narrow_tansform.show(5)
# df_narrow_tansform.explain()

+------------+--------+---------------+--------------------+------+----+----------+--------------+----------------+-------+--------+--------------------+---------+
|passenger_id|survived|passenger_class|                name|   sex|age |sib_spouse|parerent_child|         tickets|   fare|embarked|          first_name|last_name|
+------------+--------+---------------+--------------------+------+----+----------+--------------+----------------+-------+--------+--------------------+---------+
|           1|       0|              3|Braund, Mr. Owen ...|  male|22.0|         1|             0|       A/5 21171|   7.25|       S|     Mr. Owen Harris|   Braund|
|           2|       1|              1|Cumings, Mrs. Joh...|female|38.0|         1|             0|        PC 17599|71.2833|       C|Mrs. John Bradley...|  Cumings|
|           3|       1|              3|Heikkinen, Miss. ...|female|26.0|         0|             0|STON/O2. 3101282|  7.925|       S|         Miss. Laina|Heikkinen|
|           4|  

In [17]:
df_narrow_tansform.explain()

== Physical Plan ==
*(1) Project [passenger_id#306, survived#307, passenger_class#308, name#309, sex#310, age #311, sib_spouse#312, parerent_child#313, tickets#314, fare#315, embarked#316, split(name#309, , , -1)[1] AS first_name#455, split(name#309, , , -1)[0] AS last_name#468]
+- *(1) Scan ExistingRDD[passenger_id#306,survived#307,passenger_class#308,name#309,sex#310,age #311,sib_spouse#312,parerent_child#313,tickets#314,fare#315,embarked#316]


In [ ]:
ndf =  df_narrow_tansform.drop('name')
ndf.show()
ndf.printSchema()


In [ ]:
# Re-Arranging The Columns
nndf =  ndf.select('passenger_id', 'first_name', 'last_name', 'passenger_class', 'survived','sex', 'age ', 'tickets', 'fare')
# nndf.explain()

In [42]:
# Create A Temporary View / Table From The Existing DataFrame.
ndf.createOrReplaceTempView("titanic")


In [48]:
spark.sql('select max(t.fare) from titanic t').take(2)

[Row(max(fare)=512.3292236328125)]

In [65]:
# Determine the Total Number Of Survived Passengers.
spark.sql('select count(survived) from titanic where survived = 1;').show()

+---------------+
|count(survived)|
+---------------+
|            290|
+---------------+


In [67]:
# Select Top 5 passenger class, where the survival rate is high.
spark.sql('select passenger_class, count(survived) AS Survived from titanic where survived = 1 group by passenger_class order by Survived DESC;').show()

+---------------+--------+
|passenger_class|Survived|
+---------------+--------+
|              1|     122|
|              3|      85|
|              2|      83|
+---------------+--------+


In [60]:
# Select Survival Rate based on Sex.
spark.sql('select sex, count(survived) AS Survived from titanic where survived = 1 group by sex order by Survived DESC;').show()

+------+--------+
|   sex|Survived|
+------+--------+
|female|     197|
|  male|      93|
+------+--------+
